In [2]:
import pathlib
from typing import List

import gym
import gym_checkerboard  # noqa: F401
import lcs.agents.racs as racs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from lcs.agents.racs.metrics import count_averaged_regions
from lcs.metrics import population_metrics
from lcs.representations.RealValueEncoder import RealValueEncoder

from src.decorators import repeat, get_from_cache_or_run
from src.visualization import PLOT_DPI

plt.ioff()  # turn off interactive plotting

cache_dir = f'{pathlib.Path().absolute()}/cache'
plot_dir = f'{pathlib.Path().absolute()}/plots'


def encode(p, bits):
    return int(RealValueEncoder(bits).encode(p))


def metrics_to_df(metrics: List) -> pd.DataFrame:
    lst = [[
        d['trial'],
        d['population'],
        d['reliable'],
        d['reward'],
        d['regions'][1],
        d['regions'][2],
        d['regions'][3],
        d['regions'][4],
    ] for d in metrics]

    df = pd.DataFrame(lst, columns=['trial', 'population', 'reliable', 'reward', 'region_1', 'region_2', 'region_3',
                                    'region_4'])
    df = df.set_index('trial')
    df['phase'] = df.index.map(lambda t: "explore" if t % 2 == 0 else "exploit")

    return df


def average_experiment_runs(runs_dfs: List[pd.DataFrame]) -> pd.DataFrame:
    return pd.concat(runs_dfs).groupby(['trial', 'phase']).mean().reset_index(level='phase')


def single_experiment(env_provider, encoder_bits, trials):
    env = env_provider()
    env.reset()

    def _metrics(agent, environment):
        population = agent.population
        metrics = {
            'regions': count_averaged_regions(population)
        }

        # Add basic population metrics
        metrics.update(population_metrics(population, environment))
        return metrics

    cfg = racs.Configuration(
        classifier_length=env.observation_space.shape[0],
        number_of_possible_actions=env.action_space.n,
        encoder=RealValueEncoder(encoder_bits),
        user_metrics_collector_fcn=_metrics,
        epsilon=0.9,
        do_ga=True,
        theta_r=0.9,
        theta_i=0.3,
        theta_ga=100,
        cover_noise=0.1,
        mutation_noise=0.25,
        chi=0.6,
        mu=0.2)

    # create agent
    agent = racs.RACS(cfg)

    # run computations
    metrics = agent.explore_exploit(env, trials)

    return metrics_to_df(metrics)


def plot_condition_interval_regions(df, window=10, plot_filename=None):
    fig, ax = plt.subplots()

    df['region_1'].rolling(window=window).mean().plot(label='Region 1 [pi, qi)', ax=ax)
    df['region_2'].rolling(window=window).mean().plot(label='Region 2 [pmin, qi)', ax=ax)
    df['region_3'].rolling(window=window).mean().plot(label='Region 3 [pi, qmax)', ax=ax)
    df['region_4'].rolling(window=window).mean().plot(label='Region 4 [pmin, qmax)', ax=ax)

    ax.set_title('Condition Interval Regions')
    ax.set_xlabel('Trial')
    ax.set_ylabel('Proportion')
    plt.legend()

    if plot_filename:
        fig.savefig(plot_filename, dpi=PLOT_DPI, bbox_inches='tight')


def plot_population(df, window=10, plot_filename=None):
    fig, ax = plt.subplots()

    df['population'].rolling(window=window).mean().plot(label='macroclassifiers', ax=ax)
    df['reliable'].rolling(window=window).mean().plot(label='reliable', ax=ax)

    ax.set_title('Classifier numerosity')
    ax.set_xlabel('Trial')
    ax.set_ylabel('Number of classifiers')
    ax.set_yscale('log')
    plt.legend()

    if plot_filename:
        fig.savefig(plot_filename, dpi=PLOT_DPI, bbox_inches='tight')


def plot_performance(df, window=50, plot_filename=None):
    fig, ax = plt.subplots()

    explore_df = df[df['phase'] == 'explore']
    exploit_df = df[df['phase'] == 'exploit']

    explore_df['reward'].rolling(window=window).mean().plot(label='explore', ax=ax)
    exploit_df['reward'].rolling(window=window).mean().plot(label='exploit', ax=ax)
    plt.axhline(1.0, c='black', linestyle=':')

    ax.set_title('Performance (average reward)')
    ax.set_xlabel('Trial')
    ax.set_ylabel('Reward')
    ax.set_ylim([.4, 1.05])
    plt.legend()

    if plot_filename:
        fig.savefig(plot_filename, dpi=PLOT_DPI, bbox_inches='tight')

# Experiment 2 - Nature of the intervals
Info about algorithms, environments used, metrics collected, methods, reproducibility, statistical verification


## Setup
To highlight the impact of selecting proper bit number for the encoding the following table highlights the number of distinct values for each perception step (increased by `0.1`).

In [3]:
r = np.arange(0, 1.1, .1)

df = pd.DataFrame(r, columns=['Perception'])

for bit in [1, 2, 3, 4, 5, 6, 7]:
    df[f'{bit}-bit'] = df.apply(lambda row: encode(row['Perception'], bit), axis=1)

df.set_index('Perception', inplace=True)
df

,1-bit,2-bit,3-bit,4-bit,5-bit,6-bit,7-bit
Perception,,,,,,,
0.0,0,0,0,0,0,0,0
0.1,0,0,0,1,3,6,12
0.2,0,0,1,3,6,12,25
0.3,0,1,2,4,9,19,38
0.4,0,1,3,6,12,25,51
0.5,1,2,4,8,16,32,64
0.6,1,2,4,9,19,38,76
0.7,1,2,5,11,22,44,89
0.8,1,3,6,12,25,51,102


In [4]:
def encode_array(arr, bits):
    return np.fromiter((encode(x, bits=bits) for x in arr), int)


def plot_checkerboard_splits(splits, bits, points=100):
    fig = plt.figure(figsize=(12, 5))
    ax = fig.add_subplot(111)

    # Visualize splits
    for k in np.linspace(0, 1, splits + 1):
        ax.axvline(x=k, ymin=0, ymax=1, linewidth=1, linestyle=':', color='black')

    # Add some points
    x = np.random.random(points)
    y = np.random.random(points)
    colors = encode_array(x, bits)
    ax.scatter(x, y, c=colors, s=20, alpha=.8)

    for i, txt in enumerate(colors):
        ax.annotate(txt, xy=(x[i] + .005, y[i] + .005), size=8, alpha=.8)

    ax.set_title(f"{bits}-bit encoder mapping on {splits} checkerboard splits")
    ax.set_xlabel("Perception value")
    ax.set_ylim(-0.1, 1.1)
    ax.set_xlim(-0.02, 1.02)
    ax.get_yaxis().set_visible(False)

    fig.savefig(f'{plot_dir}/checkerboard_{splits}_splits_{bits}_bits.png', dpi=PLOT_DPI)

In [5]:
%%capture
plot_checkerboard_splits(splits=3, bits=4)

:::{figure-md} checkerboard-3-splits-4-bits
:class: full-width
<img src="plots/checkerboard_3_splits_4_bits.png">

Checkerboard splits
:::

## Results

In [6]:
NUM_EXPERIMENTS = 50
TRIALS = 15_000
USE_RAY = True


def checkboard_env_provider():
    import gym_checkerboard  # noqa: F401
    return gym.make('checkerboard-2D-3div-v0')


@get_from_cache_or_run(cache_path=f'{cache_dir}/checkerboard_3x3/encoding_4bit.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def run_checkerboard_4bit_encoding():
    return single_experiment(checkboard_env_provider, encoder_bits=4, trials=TRIALS)


# run computations
checkerboard_encoding_4bit_runs = run_checkerboard_4bit_encoding()

# average runs
checkerboard_encoding_4bit_avg = average_experiment_runs(checkerboard_encoding_4bit_runs)

2022-01-19 19:39:04,801	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


In [1]:
%%capture

# generate plots
plot_condition_interval_regions(checkerboard_encoding_4bit_avg,
                                plot_filename=f'{plot_dir}/checkerboard_4bit_regions.png')
plot_performance(checkerboard_encoding_4bit_avg, plot_filename=f'{plot_dir}/checkerboard_4bit_performance.png')
plot_population(checkerboard_encoding_4bit_avg, plot_filename=f'{plot_dir}/checkerboard_4bit_population.png')

NameError: name 'plot_condition_interval_regions' is not defined

:::{figure-md} checkerboard4bit-enc4bit-regions-fig
<img src="plots/checkerboard_4bit_regions.png">

Condition interval regions in 3x3 Checkerboard environment.
:::

:::{figure-md} checkerboard4bit-enc4bit-performance-fig
<img src="plots/checkerboard_4bit_performance.png">

Performance of 3x3 Checkerboard environment.
:::

:::{figure-md} checkerboard4bit-enc4bit-population-fig
<img src="plots/checkerboard_4bit_population.png">

Population of 3x3 Checkerboard environment.
:::

## Statistical verification

```{admonition} Hypothesis testing
:class: tip
I don't know if any hypothesis tests are needed here. The main value in my opinion is in Figure {numref}`{number} <checkerboard4bit-enc4bit-regions-fig>`. What might be beneficial is to plot what are the interval propotions wrt to the encoding bits used. But that would require some clever visualization (or a table).
```

## Observations
What conclusions can be drawn?